# RAJA Performance Suite - Base_CUDA to Base_SYCL

To implement our `Base_SYCL` variant we ported from the existing `Base_CUDA` variant.  The `Base_CUDA` variant for the `DAXPY` kernel is given.
```c
namespace rajaperf
{
namespace basic
{

  //
  // Define thread block size for CUDA execution
  //
  const size_t block_size = 256;


#define DAXPY_DATA_SETUP_CUDA \
  allocAndInitCudaDeviceData(x, m_x, iend); \
  allocAndInitCudaDeviceData(y, m_y, iend);

#define DAXPY_DATA_TEARDOWN_CUDA \
  getCudaDeviceData(m_y, y, iend); \
  deallocCudaDeviceData(x); \
  deallocCudaDeviceData(y);

__global__ void daxpy(Real_ptr y, Real_ptr x,
                      Real_type a,
                      Index_type iend)
{
   Index_type i = blockIdx.x * blockDim.x + threadIdx.x;
   if (i < iend) {
     DAXPY_BODY;
   }
}


void DAXPY::runCudaVariant(VariantID vid)
{
  const Index_type run_reps = getRunReps();
  const Index_type ibegin = 0;
  const Index_type iend = getRunSize();

  DAXPY_DATA_SETUP;

  if ( vid == Base_CUDA ) {

    DAXPY_DATA_SETUP_CUDA;

    startTimer();
    for (RepIndex_type irep = 0; irep < run_reps; ++irep) {

      const size_t grid_size = RAJA_DIVIDE_CEILING_INT(iend, block_size);
      daxpy<<<grid_size, block_size>>>( y, x, a,
                                        iend );

    }
    stopTimer();

    DAXPY_DATA_TEARDOWN_CUDA;

  } else if ( vid == RAJA_CUDA ) {

    // We will look at this in notebook 3

  } else {
     std::cout << "\n  DAXPY : Unknown Cuda variant id = " << vid << std::endl;
  }
}
```

## Data Setup
The `DAXPY_DATA_SETUP_CUDA` macro calls `allocAndInitCudaDeviceData` which is used to simplify the memory calls.

```c
  cudaMalloc( (void**)&dptr,
              len * sizeof(typename std::remove_pointer<T>::type) );
              
  cudaMemcpy( dptr, hptr,
              len * sizeof(typename std::remove_pointer<T>::type),
              cudaMemcpyHostToDevice );
                          
```
Using the USM features introduced in the SYCL 2020 provisional spec we can implement very similar functionality. We can use the USM API `sycl::malloc_device` to allocate space on the device.  `sycl::malloc_device` returns the pointer to the newly allocated device memory and doesn't pass the device pointer as an argument.  After the size argument we need to pass `sycl::malloc_device` either the device and context or the queue.  In this code we have a public static class member holding our queue named `qu`.

```c
  dptr = cl::sycl::malloc_device<typename std::remove_pointer<T>::type>(len, qu);
```

The memcpy call for SYCL is similar but it is a function member of the queue, `qu.memcpy`.  The memcpy call is asynchronous so we will wait on the returned event to ensure the memory is where we need it.

```c
  auto e = qu.memcpy( dptr, hptr,
                      len * sizeof(typename std::remove_pointer<T>::type));
  e.wait();
```

We implement this data setup specific to the `DAXPY` kernel below.

In [ ]:
%%writefile ./src/RAJAPerf/src/basic/DAXPY_Setup.hpp
x = cl::sycl::malloc_device<Real_type>(iend, qu);
y = cl::sycl::malloc_device<Real_type>(iend, qu);
auto e = qu.memcpy(x, m_x, iend * sizeof(Real_type));
auto e2 = qu.memcpy(y, m_y, iend * sizeof(Real_type));
// Wait for memcpys to finish
e.wait();
e2.wait();


## Kernel
The CUDA kernel launch is defining a grid size and block size.
```c
      const size_t grid_size = RAJA_DIVIDE_CEILING_INT(iend, block_size);
      daxpy<<<grid_size, block_size>>>( y, x, a,
                                        iend );
```
In SYCL we can use `sycl::nd_range`, which takes in two arguments, the global iteration size and the block size.  We calculate our global size to be a multiple of the block size that fits our iteration space.   The kernel looks very similar expect that we can include it inline with our kernel launch.

```c
  const size_t global_size = block_size * RAJA_DIVIDE_CEILING_INT(iend, block_size);

  qu.submit([&] (cl::sycl::handler& h) {
    h.parallel_for<class DAXPY>(cl::sycl::nd_range<1>(global_size, block_size),
                                [=] (cl::sycl::nd_item<1> item ) {

      Index_type i = item.get_global_id(0);
      // We could also calculate the global index
      //Index_type i = item.get_group(0) * item.get_local_range(0) + item.get_local_id(0);
      if (i < iend) {
        DAXPY_BODY
      }
                                    
    });
  });
```

Within the kernel, we access our index through the `sycl::nd_item` object. We  access our global index through `item.get_global_id`, we could also access our global index by calculating directly from our group and local index information. 


In [ ]:
%%writefile ./src/RAJAPerf/src/basic/DAXPY_Kernel.hpp
      const size_t global_size = block_size * RAJA_DIVIDE_CEILING_INT(iend, block_size);

      qu.submit([&] (cl::sycl::handler& h) {
        h.parallel_for(cl::sycl::nd_range<1>(global_size, block_size),
                       [=] (cl::sycl::nd_item<1> item ) {

          Index_type i = item.get_global_id(0);
          // We could also calculate the global index
          // Index_type i = item.get_group(0) * item.get_local_range(0) + item.get_local_id(0);
          if (i < iend) {
            DAXPY_BODY
          }

        });
      });

## Data Teardown
The `DAXPY_DATA_TEARDOWN_CUDA` macro calls `getCudaDeviceData` and `deallocCudaDeviceData` which are wrappers for:
```c
  cudaMemcpy( hptr, dptr,
              len * sizeof(typename std::remove_pointer<T>::type),
              cudaMemcpyDeviceToHost );
  // and
  cudaFree( dptr );
              
```

Using the SYCL 2020 provisional spec the `memcpy` looks the same as when we transfered the data to the device, except that we will switch the desitination and source arguments.  After we wait for the memcpy to finish we will free the device memory with `sycl::free`.

```c
  auto e = qu.memcpy( dptr, hptr,
                      len * sizeof(typename std::remove_pointer<T>::type));
  e.wait();

  cl::sycl::free(dptr, qu);
```

We implement this specifically for the `DAXPY` kernel below

In [ ]:
%%writefile ./src/RAJAPerf/src/basic/DAXPY_Teardown.hpp
auto e3 = qu.memcpy(m_y, y, iend * sizeof(Real_type));
// Wait for memcpy to finish
e3.wait();
cl::sycl::free(x, qu);
cl::sycl::free(y, qu);
                

## When we put it all together
```
namespace rajaperf
{
namespace basic
{

  //
  // Define thread block size for SYCL execution
  //
  const size_t block_size = 256; // We could query our device for this

#define DAXPY_DATA_SETUP_SYCL                            \
  x = cl::sycl::malloc_device<Real_type>(iend, qu);      \
  y = cl::sycl::malloc_device<Real_type>(iend, qu);      \
  auto e = qu.memcpy(x, m_x, iend * sizeof(Real_type));  \
  auto e2 = qu.memcpy(y, m_y, iend * sizeof(Real_type)); \
  e.wait(); \
  e2.wait();


#define DAXPY_DATA_TEARDOWN_SYCL                        \
  auto e = qu.memcpy(m_x, x, iend * sizeof(Real_type)); \
  e.wait();                                             \
  cl::sycl::free(x, qu);                                \
  cl::sycl::free(y, qu);
                

void DAXPY::runSyclVariant(VariantID vid)
{
  const Index_type run_reps = getRunReps();
  const Index_type ibegin = 0;
  const Index_type iend = getRunSize();

  DAXPY_DATA_SETUP; // This sets up our host data. m_x, m_y.

  if ( vid == Base_SYCL ) {
    { // Create a scope for our buffers

      DAXPY_DATA_SETUP_SYCL;

      startTimer();
      for (RepIndex_type irep = 0; irep < run_reps; ++irep) {

        const size_t global_size = block_size 
                                   * RAJA_DIVIDE_CEILING_INT(iend, block_size);

        qu.submit([&] (cl::sycl::handler& h) {
          h.parallel_for(cl::sycl::nd_range<1>{global_size, block_size},
                         [=] (cl::sycl::nd_item<1> item ) {

            Index_type i = item.get_global_id(0);
            if (i < iend) {
              DAXPY_BODY
            }

          });
        });
      }
      qu.wait(); // Wait for computation to finish before stopping timer
      stopTimer();
     
    } // End of buffer scope

    DAXPY_DATA_TEARDOWN_SYCL;
 
  } else if ( vid == RAJA_SYCL ) {

  // We will do this later

  } else {
     std::cout << "\n  DAXPY : Unknown Sycl variant id = " << vid << std::endl;
  }

}

} // end namespace basic
} // end namespace rajaperf
```

### Now lets rebuild the performance suite

In [1]:
!qsub build_RAJAPerf

/bin/sh: Build: command not found


### After it finishes building, lets run the performance suite with our `Base_SYCL DAXPY` kernel

In [ ]:
!qstat

In [ ]:
!qsub -l nodes=1:gpu:ppn=2 run_RAJAPerf

### After the run finishes, lets check the report

In [ ]:
!cat output/RAJAPerf-timing.csv